# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp dash_components

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
except: 
    from load_data import *
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import json
from git import Repo
import pandas as pd
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, html, ctx  # pip install dash
import dash_ag_grid as dag
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import random
import dash_mantine_components as dmc
import numpy as np

In [ ]:
# TODO - do a need a state class that tracks the state of different website objects to help with
# updating etc. 
# TODO - not even state, should it be an object that contains everythin to track. Execpt if I am patching,
# then the tracking in mem is unessary, so better to just track necesary values like: last changes card etc. 

## Load Data

In [ ]:
sol_geo = SolomonGeo.load_pickle("/testData/")

## Build the Cloreopath Map

In [ ]:
#| export
# TODO I should build figures and maps in another script
def define_map(sol_df:SolomonGeo # Solomon geo object containing census data to input into map
                )->type(go.Figure()): # Returns a graph object figure
    '''
    Creates and returns the base cloreopath map
    '''
    # TODO - should I update this into a class with methods for updating
    # the other things? Acutally maybe as another function if the update is done through patch
    
    # cols_dd dictates the aggregation that will be visable
    cols_dd = sol_df.geo_levels
    # define traces and buttons at once
    traces = []
    # TODO if fails remember I changed visible from cols_dd
    for value in cols_dd:
        z_vals = sol_df.get_census(geo_filter = value, var = 'Key Statistics', 
                                                 measure = 'Total Households').values
        z_vals = z_vals.reshape((z_vals.shape[0],))
        traces.append(go.Choroplethmapbox(
                                geojson=sol_df.get_geojson(geo_filter = value),
                               locations=sol_df.locations[value],
                               customdata = sol_df.locations[value],
                               # TODO undo hardcoding
                               z = z_vals,
                               zmin = np.min(z_vals),
                               zmax = np.min(z_vals),
                               colorscale="deep",
                                marker_line_width = 0.5,
                                #zauto=True,
                                selectedpoints=None,
                                hovertemplate = '%{customdata} <extra>%{z}</extra>',
                visible= True if value==cols_dd[0] else False))
        
    # Show figure
    fig = go.Figure(data=traces)
    # This is in order to get the first title displayed correctly
    first_title = cols_dd[0]
    fig.update_layout(title=f"<b>{first_title}</b>",
                        title_x=0.5,
                        mapbox_style = 'carto-positron',
                        mapbox_zoom = 5,
                        mapbox_center={"lat": -9.565766, "lon": 162.012453},
                        margin={"r":0,"t":0,"l":0,"b":0},
    )
    
    return fig


go.

In [ ]:
#| hide
show_doc(define_map)

---

### define_map

>      define_map (sol_df:SolomonIslandsDataMap.load_data.SolomonGeo)

*Creates and returns the base cloreopath map*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| sol_df | SolomonGeo | Solomon geo object containing census data to input into map |
| **Returns** | **Figure** | **Returns a graph object figure** |

In [ ]:
z_vals = sol_geo.get_census(geo_filter = 'Province', var = 'Key Statistics', 
                                                 measure = 'Total Households').values
np.min(z_vals)

688

In [ ]:
fig = define_map(sol_geo)


In [ ]:
print(fig['data'][0]['zmin'])

688


### Election Maps
Slightly different as they have less geographies and colour based on winning party

In [ ]:
sol_geo.elec_wide.head()

,Type,Year,Geo,loc_name,0,1,2,Winning Party
0,National Parliament,2024,Constituency,Auki-Langalanga,Cooper Wale (SID) - 4314 (66.2%),David Filia (OUR) - 1348 (20.7%),Talauburi Anisi (KAD) - 488 (7.5%),SID
1,National Parliament,2024,Constituency,Baegu-Asifola,Tagini Makario (OUR) - 2391 (37.4%),Celsus Talifilu (U4C) - 2147 (33.6%),Douglas Buga (SIP) - 1009 (15.8%),OUR
2,National Parliament,2024,Constituency,Central Guadalcanal,Shanel Agovaka (OUR) - 3454 (43.1%),Manengelea Rex (IND) - 1592 (19.8%),Denson Denni (KAD) - 1381 (17.2%),OUR
3,National Parliament,2024,Constituency,Central Honiara,Darcy Lilo (SIP) - 8223 (63.4%),Alfred Efona (UP) - 3632 (28.0%),Francis Idu (OUR) - 669 (5.2%),SIP
4,National Parliament,2024,Constituency,Central Kwara'ae,Ricky Fuo'o (UP) - 5208 (47.9%),Jackson Fiulaua (OUR) - 1831 (16.8%),Young Wate (IND) - 1537 (14.1%),UP


In [ ]:
sol_geo.elec_wide["Winning Party"].unique()

array(['SID', 'OUR', 'SIP', 'UP', 'IND', 'PFP', 'U4C', 'KAD', 'DAP'],
      dtype=object)

In [ ]:
sol_geo.elec["Party"].unique()

array(['UP', 'PLD', 'KAD', 'OUR', 'SID', 'IND', 'SIP', 'U4C', 'PAP',
       'PFP', 'DAP', 'NTP', 'GPS'], dtype=object)

# TODO
I need to make a unique color for each party, then colorscale will need to have all as an option. Probably a dictionary is better.
Use the color brewer colors plus maybe another colour like black https://colorbrewer2.org/#type=qualitative&scheme=Set3&n=12


In [ ]:
#| export
def election_map(sol_df:SolomonGeo # Solomon geo object containing census data to input into map
                )->type(go.Figure()): # Returns a graph object figure
    '''
    Creates and returns a chloreopath map for election
    '''
    # define traces and buttons at once
    traces = []

    for i, winner in enumerate(sol_df.elec_wide["Winning Party"].unique()):
        dfp = sol_df.elec_wide[sol_df.elec_wide["Winning Party"] == winner]
        traces.append(go.Choroplethmapbox(
                                geojson=sol_df.get_geojson(geo_filter = "Constituency"),
                               locations=dfp.loc_name,
                               customdata = np.stack((dfp.loc_name, dfp["Winning Party"], dfp.iloc[:, 4]), axis = 1), # Stack to make it display in the right order # TODO change column names so I don't need iloc
                               # TODO undo hardcoding
                               z = [i, ] * len(dfp),
                               colorscale=sol_df.colorscales[i],
                               showscale = False,
                                marker_line_width = 0.5,
                                #zauto=True,
                                selectedpoints=None,
                                hovertemplate = '%{customdata[0]} <extra><b>Winning Party</b>: %{customdata[1]}<br><b>Candidate</b>: %{customdata[2]}</extra>',
                                legend = "legend1",
                                showlegend = True,
                                legendgroup = winner,
                                name = winner,
                #visible= True if value==cols_dd[0] else False,
                ))
        
    # Show figure
    fig = go.Figure(data=traces)
    # This is in order to get the first title displayed correctly
    #first_title = cols_dd[0]
    fig.update_layout(#title=f"<b>{"Constituency"}</b>",
                        title_x=0.5,
                        mapbox_style = 'carto-positron',
                        mapbox_zoom = 5,
                        mapbox_center={"lat": -9.565766, "lon": 162.012453},
                        margin={"r":0,"t":0,"l":0,"b":0},
                        showlegend = True,
                        legend_title = "Party",
    )
    
    return fig


In [ ]:
#election_map(sol_geo).show()

In [ ]:
election_map(sol_geo)['data'][0]['colorscale']

((0.0, '#8dd3c7'), (1.0, '#8dd3c7'))

## Create a bar graph to display current data

In [ ]:
#| export
def gen_bar_plot(sol_geo:SolomonGeo, # Solomon geo object containing census data to input into map
                    geo_filter:str, # The desired aggregation of the geography
                    variable:str, # The variable to use to create the bar plot
                    measure:str, # The measure to highlight on the bar graph
                    locations:[str] = [], # Desired location within aggregation
                    type_filter:str = 'Total', # The type aggregartion
                )->type(go.Figure()): # Returns a graph object figure of a barplot
    '''Create a bar plot that show the census selected census data'''
    figtext = 'Showing ' + variable + ' for '
    if locations == []:
        df = sol_geo.get_census(geo_filter, variable, type_filter = type_filter, agg = True)
        df = pd.DataFrame(df).transpose()
        df.index = ['Total']
        locations = ['Total']
        figtext += 'Solomon Islands'
    else:
        df = sol_geo.get_census(geo_filter, variable, loc_filter = locations, type_filter = type_filter)
        figtext += ', '.join(locations)
    fig = go.Figure()
    measures = list(df.columns)

    for loc in locations:
        displayText = df.loc[df.index == loc].values[0]
        if type_filter == 'Proportion': 
            displayText = np.round(displayText * 100, decimals = 2)
            displayText = [item + "%" for item in displayText.astype(str)]
        fig.add_trace(go.Bar(
            x = measures,
            y = df.loc[df.index == loc].values[0],
            name = loc,
            customdata = np.repeat(loc, len(measures)),
            hovertemplate = '%{customdata} <extra>%{x}<br><b>%{y}</extra>',
            text = displayText,
            textposition='auto',
        ))
    # TODO create dynamic text with Location name and Variable
    # TODO add standout text
    # TODO should this be ordered? Hinders comparison. Can I order the dataset somewhere else?
    fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text=figtext
                      , xaxis={'categoryorder':'total descending'})
    fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
    if type_filter == 'Proportion':
        '''If proportion, display proportion!'''
        fig.update_layout(yaxis_tickformat='.1%')
        fig.update_layout(hovermode="x unified")
    return fig

In [ ]:
#| hide
show_doc(gen_bar_plot)

---

### gen_bar_plot

>      gen_bar_plot (sol_geo:SolomonIslandsDataMap.load_data.SolomonGeo,
>                    geo_filter:str, variable:str, measure:str,
>                    locations:[<class'str'>]=[], type_filter:str='Total')

*Create a bar plot that show the census selected census data*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sol_geo | SolomonGeo |  | Solomon geo object containing census data to input into map |
| geo_filter | str |  | The desired aggregation of the geography |
| variable | str |  | The variable to use to create the bar plot |
| measure | str |  | The measure to highlight on the bar graph |
| locations | [<class 'str'>] | [] | Desired location within aggregation |
| type_filter | str | Total | The type aggregartion |
| **Returns** | **Figure** |  | **Returns a graph object figure of a barplot** |

In [ ]:
gen_bar_plot(sol_geo, 'Ward', "Main source of household income in last 12 months", 'Wages Salary', ['Cruz', 'Vavaea'], 'Proportion')

### Bar Plot for Elections Data
Example plot for Auki-Langalanga

In [ ]:
df = sol_geo.elec
loc = 'Auki-Langalanga'

df.loc[df['loc_name'] == loc, 'candParty'].values

array(['Joseph Waleanisia (UP)', 'Obadiah Koti (PLD)',
       'Talauburi Anisi (KAD)', 'David Filia (OUR)', 'Cooper Wale (SID)'],
      dtype=object)

In [ ]:
fig = go.Figure()
measures = list(df.columns)
fig.add_trace(go.Bar(
    x = df.loc[df['loc_name'] == loc, 'Candidate'].values,
    y = df.loc[df['loc_name'] == loc, 'Total'].values,
    name = loc,
    marker = dict(color = list(map(lambda y: sol_geo.colormap[y], df.loc[df['loc_name'] == loc, 'Party'].values))),
    hovertemplate = '%{x} <extra>%{y}</extra>',
    text = df.loc[df['loc_name'] == loc, 'Total'].values,
    textposition='auto',
))
fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text="Title",
                   xaxis={'categoryorder':'total descending'}, )
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

In [ ]:
#| export
def election_bar_plot(sol_geo:SolomonGeo, # Solomon geo object containing census data to input into map
                    geo_filter:str = "Constituency", # The desired aggregation of the geography
                    year:int = 2024, # The year of the elections data
                    election:str = 'National Parliament', # The type of election.	
                    location:str = 'Auki-Langalanga', # Desired location within aggregation
                    type_filter:str = 'Total', # The type of data, eitehr Total of Proportion
                )->type(go.Figure()): # Returns a graph object figure of a barplot
    '''Create a bar plot that show the census selected election data'''
    df = sol_geo.elec

    df = df.loc[(df['loc_name'] == location) & (df['Type'] == election)  & (df['Year'] == year) & (df['Geo'] == geo_filter), :]

    figtext = str(year) + " " + election + ' Election Results in ' + location

    fig = go.Figure()
    displayText = df.loc[df['loc_name'] == location, type_filter].values
    if type_filter == 'Proportion': 
        # Turn into % when it's a proportion
        displayText = np.round(displayText * 100, decimals = 2)
        displayText = [item + "%" for item in displayText.astype(str)]
    fig.add_trace(go.Bar(
        x = df.loc[df['loc_name'] == location, 'candParty'].values,
        y = df.loc[df['loc_name'] == location, type_filter].values,
        name = location,
        marker = dict(color = list(map(lambda y: sol_geo.colormap[y], df.loc[df['loc_name'] == location, 'Party'].values))),
        hovertemplate = '%{x} <extra>%{y}</extra>',
        text = displayText,
        textposition='auto',
    ))
    fig.update_layout(barmode='group', xaxis_tickangle=-45, title_text=figtext,
                    xaxis={'categoryorder':'total descending'})
    fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
    if type_filter == 'Proportion':
        '''If proportion, display proportion!'''
        fig.update_layout(yaxis_tickformat='.1%')
        fig.update_layout(hovermode="x unified")
    return fig

In [ ]:
show_doc(election_bar_plot)

---

### election_bar_plot

>      election_bar_plot (sol_geo:SolomonIslandsDataMap.load_data.SolomonGeo,
>                         geo_filter:str='Constituency', year:int=2024,
>                         election:str='National Parliament',
>                         location:str='Auki-Langalanga',
>                         type_filter:str='Total')

*Create a bar plot that show the census selected election data*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sol_geo | SolomonGeo |  | Solomon geo object containing census data to input into map |
| geo_filter | str | Constituency | The desired aggregation of the geography |
| year | int | 2024 | The year of the elections data |
| election | str | National Parliament | The type of election. |
| location | str | Auki-Langalanga | Desired location within aggregation |
| type_filter | str | Total | The type of data, eitehr Total of Proportion |
| **Returns** | **Figure** |  | **Returns a graph object figure of a barplot** |

In [ ]:
election_bar_plot(sol_geo, type_filter = 'Proportion')

## Create Data Tables
### Census Data Table
A method that creates data tables for census

In [ ]:
#| export
# TODO should this method be appended to sol_geo??
def gen_census_grid(sol_geo:SolomonGeo, # Solomon geo object containing census data to input into map
                    geo_filter:str, # The desired aggregation of the geography
                    variable:str, # The variable to use to create the bar plot
                    measure:str, # The measure to highlight on the bar graph
                    locations:[str] = None, # Desired location within aggregation
                    type_filter:str = 'Total', # The type aggregartion
                    grid_rows:int = 10, # The number of rows to display
                )->dag.AgGrid: # Returns a graph object figure of a barplot
    '''Creates a basic data table using dash grid'''
    figtext = 'Showing ' + variable + ' by ' + geo_filter
    df = sol_geo.get_census(geo_filter, variable, loc_filter = locations, type_filter = type_filter)
    df.insert(0, geo_filter, df.index) # Put geo locations at the front
    
    # pre define the column definitions, with extra speficiations for the locations
    colDef = [{"field": geo_filter, "headerName": geo_filter, "filter": True, "lockPinned": True, "cellClass": "lock-pinned"}]                    
    colDef += [{"field": i, "headerName": i} for i in df.columns[1:]]
    dt = dag.AgGrid(
        id = 'dash-grid',
        rowData = df.to_dict('records'),
        columnDefs = colDef,
        columnSize="sizeToFit",
        defaultColDef={"resizable": True, "sortable": True, "minWidth": 100,},
        dashGridOptions={"pagination": True, "domLayout": "autoHeight", "paginationPageSize": grid_rows, "wrapHeaderText": True,  "autoHeaderHeight": True},
        style={"height": None},
        csvExportParams={
                "fileName": "Solomons 2009 Census Data " + variable + " by " + geo_filter + " - " + type_filter + ".csv",
            },
    )

    return dt

In [ ]:
show_doc(gen_census_grid)

---

### gen_census_grid

>      gen_census_grid (sol_geo:SolomonIslandsDataMap.load_data.SolomonGeo,
>                       geo_filter:str, variable:str, measure:str,
>                       locations:[<class'str'>]=None, type_filter:str='Total',
>                       grid_rows:int=10)

*Creates a basic data table using dash grid*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sol_geo | SolomonGeo |  | Solomon geo object containing census data to input into map |
| geo_filter | str |  | The desired aggregation of the geography |
| variable | str |  | The variable to use to create the bar plot |
| measure | str |  | The measure to highlight on the bar graph |
| locations | [<class 'str'>] | None | Desired location within aggregation |
| type_filter | str | Total | The type aggregartion |
| grid_rows | int | 10 | The number of rows to display |
| **Returns** | **AgGrid** |  | **Returns a graph object figure of a barplot** |

In [ ]:
gen_census_grid(sol_geo, 'Province', "Main source of household income in last 12 months", 'Wages Salary')

AgGrid(id='dash-grid', columnDefs=[{'field': 'Province', 'headerName': 'Province', 'filter': True, 'lockPinned': True, 'cellClass': 'lock-pinned'}, {'field': 'No income', 'headerName': 'No income'}, {'field': 'Wages Salary', 'headerName': 'Wages Salary'}, {'field': 'Own business', 'headerName': 'Own business'}, {'field': 'Sale fish crop craft', 'headerName': 'Sale fish crop craft'}, {'field': 'Land lease', 'headerName': 'Land lease'}, {'field': 'House rent', 'headerName': 'House rent'}, {'field': 'Remittances', 'headerName': 'Remittances'}, {'field': 'Other source', 'headerName': 'Other source'}], columnSize='sizeToFit', csvExportParams={'fileName': 'Solomons 2009 Census Data Main source of household income in last 12 months by Province - Total.csv'}, dashGridOptions={'pagination': True, 'domLayout': 'autoHeight', 'paginationPageSize': 10, 'wrapHeaderText': True, 'autoHeaderHeight': True}, defaultColDef={'resizable': True, 'sortable': True, 'minWidth': 100}, rowData=[{'Province': 'Cent

### Create a Population Grid

In [ ]:
#| export
# TODO should this method be appended to sol_geo??
def gen_pop_grid(sol_geo:SolomonGeo, # Solomon geo object containing census data to input into map
                    years:str, # Selected data years
                    variable:str, # The variable to use to create the bar plot
                    measure:str, # The measure to highlight on the bar graph
                    geo_filter:str = 'Province', # The desired aggregation of the geography
                    locations:[str] = None, # Desired location within aggregation
                    type_filter:str = 'Total', # The type aggregartion
                    grid_rows:int = 10, # The number of rows to display
                ) -> dag.AgGrid: # Returns a graph object figure of a barplot
    '''Creates a basic data table using dash grid'''
    figtext = 'Showing ' + variable + ' Projected Population by ' + geo_filter
    df = sol_geo.get_pop(years, variable, measure, type_filter = type_filter,)
    # Remove hierarchical columns
    df.columns = df.columns.get_level_values(1)
    df['location'] = df.index 
    df = pd.pivot(df, index = ['location', 'year'], columns = 'Age_Bracket', values = measure)

    # Change index 
    df.insert(0, 'Year', df.index.get_level_values(1))
    df.insert(0, geo_filter, df.index.get_level_values(0))
    df = df.reindex(columns = ['Year', geo_filter, '0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34',
        '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
        '70-74', '75-79', '80+'])
    
    # pre define the column definitions, with extra speficiations for the locations
    # TODO care of hardcoding, will break in future if projections are available for other geos
    colDef = [{"field": geo_filter, "headerName": geo_filter, "filter": True, "lockPinned": True, "cellClass": "lock-pinned"}]
    colDef += [{"field": 'Year', "headerName": 'Year', "filter": True, "lockPinned": True, "cellClass": "lock-pinned"}]                    
    colDef += [{"field": i, "headerName": i} for i in df.columns[2:]]
    dt = dag.AgGrid(
        id = 'dash-grid',
        rowData = df.to_dict('records'),
        columnDefs = colDef,
        columnSize="sizeToFit",
        defaultColDef={"resizable": True, "sortable": True, "minWidth": 100,},
        dashGridOptions={"pagination": True, "domLayout": "autoHeight", "paginationPageSize": grid_rows, "wrapHeaderText": True,  "autoHeaderHeight": True},
        style={"height": None},
        csvExportParams={
                "fileName": "Solomons 2009 Population Projections for " + variable + " Population by " + geo_filter + " - " + type_filter + ".csv",
            },
        )

    return dt

In [ ]:
gen_pop_grid(sol_geo, [2023, 2024, 2025], 'Population', 'Total',)

AgGrid(id='dash-grid', columnDefs=[{'field': 'Province', 'headerName': 'Province', 'filter': True, 'lockPinned': True, 'cellClass': 'lock-pinned'}, {'field': 'Year', 'headerName': 'Year', 'filter': True, 'lockPinned': True, 'cellClass': 'lock-pinned'}, {'field': '0-4', 'headerName': '0-4'}, {'field': '5-9', 'headerName': '5-9'}, {'field': '10-14', 'headerName': '10-14'}, {'field': '15-19', 'headerName': '15-19'}, {'field': '20-24', 'headerName': '20-24'}, {'field': '25-29', 'headerName': '25-29'}, {'field': '30-34', 'headerName': '30-34'}, {'field': '35-39', 'headerName': '35-39'}, {'field': '40-44', 'headerName': '40-44'}, {'field': '45-49', 'headerName': '45-49'}, {'field': '50-54', 'headerName': '50-54'}, {'field': '55-59', 'headerName': '55-59'}, {'field': '60-64', 'headerName': '60-64'}, {'field': '65-69', 'headerName': '65-69'}, {'field': '70-74', 'headerName': '70-74'}, {'field': '75-79', 'headerName': '75-79'}, {'field': '80+', 'headerName': '80+'}], columnSize='sizeToFit', csv

dashGridOptions={'pagination': True, 'domLayout': 'autoHeight', 'paginationPageSize': 10},

## Create KPI
The kpi card is useful for displaying a particular value as a standout kpi

In [ ]:
#| export

# TODO create bottom padding

def gen_kpi(sg:SolomonGeo, # Input data object
                year:str, # Year of the kpi
                variable:str, # Variable to dispaly
                measure:str, # Measure to display
                ages:[str], # ages to display
                loc:[str] = [], # Desired location within aggregation
                type_filter:str = 'Total', # The type 
            )-> (dcc.Markdown, dcc.Markdown): # Returns a column containing a title and accordian items
    '''
    Create a list accordians for each variable, where each accordian contains a card for 
    each measure of that variable
    '''
    # TODO work out how to wrap kpi into text
    if loc == []:
        df = sg.get_pop(years = [year],
                        var = variable,
                        measure = measure, 
                        ages = ages,
                        type_filter = type_filter,
                        agg = True).values[0]
    else:
        df = sg.get_pop(years = [year], 
                        var = variable,
                        measure = measure, 
                        ages = ages,
                        loc_filter = loc,
                        type_filter = type_filter,
                        agg = True).values[0]
    if type_filter == 'Total':
        kpi = dcc.Markdown(children = "# " + str(format(int(df[0]), ",d")))
        text = dcc.Markdown(children = type_filter.lower() + " persons in current selection")
    elif type_filter == 'Proportion':
        kpi = dcc.Markdown(children = "# " + str(format(float(df[0]), ".1%")))
        text = dcc.Markdown(children = "projected population in current selection")
    else:
        raise ValueError('The type passed to the gen_kpi function must be one of the following: \'Total\', \'Proportion\'.')
    
    return kpi, text

In [ ]:
#| hide
show_doc(gen_kpi)

---

### gen_kpi

>      gen_kpi (sg:SolomonIslandsDataMap.load_data.SolomonGeo, year:str,
>               variable:str, measure:str, ages:[<class'str'>],
>               loc:[<class'str'>]=[], type_filter:str='Total')

*Create a list accordians for each variable, where each accordian contains a card for 
each measure of that variable*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sg | SolomonGeo |  | Input data object |
| year | str |  | Year of the kpi |
| variable | str |  | Variable to dispaly |
| measure | str |  | Measure to display |
| ages | [<class 'str'>] |  | ages to display |
| loc | [<class 'str'>] | [] | Desired location within aggregation |
| type_filter | str | Total | The type |
| **Returns** | **(<class 'dash.dcc.Markdown.Markdown'>, <class 'dash.dcc.Markdown.Markdown'>)** |  | **Returns a column containing a title and accordian items** |

In [ ]:
# Build and test a card
gen_kpi(sol_geo, 2022, 'Population', 'Total', ["0-4"])

(Markdown('# 83,808'), Markdown('total persons in current selection'))

## Create function to make dropdowns

In [ ]:
#| export
def gen_dd(location_list:[str], # a list of locations
           id:str, # Id of the dropdown
           place_holder:str = None, # a placeholder message to display
           val:str = None, # The starting value of the dropdown
           clear:bool = False, # pick whether the ]
           height:int = 35, # height of the dropdown text
           multi:bool = False, # Is the dropdown multi select
                        )->dcc.Dropdown: # Returns a dropdown
    '''
    Create the location dropdown from given list
    '''
    # TODO this is stupid, I am just wraping the for no reason
    # TODO is this really necessary? I anm
    dd = dcc.Dropdown(options=location_list,
                        value=val,  # initial value displayed when page first loads
                        searchable=True,
                        clearable=clear,
                        placeholder=place_holder, 
                        id = id, 
                        optionHeight=height,
                        multi=multi)
    return dd

In [ ]:
show_doc(gen_dd)

---

### gen_dd

>      gen_dd (location_list:[<class'str'>], id:str, place_holder:str=None,
>              val:str=None, clear:bool=False, height:int=35, multi:bool=False)

*Create the location dropdown from given list*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| location_list | [<class 'str'>] |  | a list of locations |
| id | str |  | Id of the dropdown |
| place_holder | str | None | a placeholder message to display |
| val | str | None | The starting value of the dropdown |
| clear | bool | False | pick whether the ] |
| height | int | 35 | height of the dropdown text |
| multi | bool | False | Is the dropdown multi select |
| **Returns** | **Dropdown** |  | **Returns a dropdown** |

## Create Population Pyramid
For use to demonstrate the population pyramid of solomon islands.
It should highlight age groups based on selection

In [ ]:
#| export
def gen_pyramid(sol_geo:SolomonGeo, # Solomon geo object containing census data to input into map
                    geo_filter:str, # The desired aggregation of the geography
                    year:str, # Selected year to display on the graph
                    variable:str = 'Population', # The variable to use to create the bar plot
                    locations:[str] = [], # Desired location within aggregation
                    type_filter:str = 'Total', # The type aggregartion
                    ages:[str] = [], # Currenly selected ages for highlighting
                )->type(go.Figure()): # Returns a population pyramid and it's title
    '''Create a population pyramid of selected population data'''
    # TODO Can't make a comparative pop pyrmid. Should I do this??
    
    # Load Data
    if locations == []:
        pop_data = sol_geo.get_pop(years = [year], var = "Population", agg = True, agg_ages = True
                                   , type_filter=type_filter)
    else:
        pop_data = sol_geo.get_pop(years = [year], var = "Population", agg = True, agg_ages = True
                                   , type_filter=type_filter, loc_filter = locations)

    # Manipulate data for pyramid
    age_df = pop_data.loc[year]
    age_df.index.name = 'age'
    age_df.drop(columns = ('Population', 'Total'))

    y_age = age_df.index
   
    # Raw values from the table (either counts or proportions in [0,1])
    male_raw = age_df[('Population', 'Males')].astype(float)
    female_raw = age_df[('Population', 'Females')].astype(float)

    # Change data depending on whether proportion or not

    if type_filter == "Proportion":
        # Work in percentage points so axis/hover show 0–100 %
        x_male = -male_raw * 100.0
        x_female = female_raw * 100.0
        axis_title = 'Population (%)'
        female_hover = '%{y}: %{x:.1f}%<extra>Female</extra>'
        male_hover = '%{y}: %{x:.1f}%<extra>Male</extra>'
        tickformat = '.1f'  # numbers like 5, 10, 15; interpreted as %
    else:
        # Use counts directly
        x_male = -male_raw
        x_female = female_raw
        axis_title = 'Population'
        female_hover = '%{y}: %{x:,.0f} people<extra>Female</extra>'
        male_hover = '%{y}: %{x:,.0f} people<extra>Male</extra>'
        tickformat = ',.0f'  # comma-separated integers

    # Create instance of the figure
    pyramid_fig = go.Figure()

    # Add traces
    pyramid_fig.add_trace(
        go.Bar(
            y=y_age,
            x=x_female,
            name='Female',
            orientation='h',
            hovertemplate=female_hover,
        )
    )

    pyramid_fig.add_trace(
        go.Bar(
            y=y_age,
            x=x_male,
            name='Male',
            orientation='h',
            hovertemplate=male_hover,
        )
    )

    # Symmetric x-axis around zero
    max_x = float(np.nanmax([abs(x_male).max(), x_female.max()]))
    max_x = max_x if max_x > 0 else 1.0

    pyramid_fig.update_layout(
        title_font_size=24,
        barmode='relative',
        bargap=0.0,
        bargroupgap=0,
        xaxis=dict(
            range=[-max_x, max_x],
            tickformat=tickformat,
            title=axis_title,
            title_font_size=14,
        ),
        yaxis=dict(title='Age Group'),
    )

    return pyramid_fig

In [ ]:
show_doc(gen_pyramid)

---

### gen_pyramid

>      gen_pyramid (sol_geo:SolomonIslandsDataMap.load_data.SolomonGeo,
>                   geo_filter:str, year:str, variable:str='Population',
>                   locations:[<class'str'>]=[], type_filter:str='Total',
>                   ages:[<class'str'>]=[])

*Create a population pyramid of selected population data*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sol_geo | SolomonGeo |  | Solomon geo object containing census data to input into map |
| geo_filter | str |  | The desired aggregation of the geography |
| year | str |  | Selected year to display on the graph |
| variable | str | Population | The variable to use to create the bar plot |
| locations | [<class 'str'>] | [] | Desired location within aggregation |
| type_filter | str | Total | The type aggregartion |
| ages | [<class 'str'>] | [] | Currenly selected ages for highlighting |
| **Returns** | **Figure** |  | **Returns a population pyramid and it's title** |

In [ ]:
gen_pyramid(sol_geo, geo_filter = 'Province', year = 2024, variable = "Population", locations = ['Western'], type_filter = "Proportion")

## Testing

In [ ]:
# TODO implementing testing, everything should at least be run here and execute and be displayed

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()